# 01 — Data Loading & Cleaning

**Objective:** Load the Open Food Facts bulk database, filter to EU countries, and clean for analysis.

## Data Source
- **Open Food Facts** bulk Parquet from Hugging Face (`openfoodfacts/product-database` → `food.parquet`)
- Full file: 4.4 GB, ~4.29M products worldwide
- Downloaded via `curl` (DuckDB remote streaming triggers HF rate limiting)
- Filtered to 27 EU countries using DuckDB, nutrients extracted from nested structs → `data/raw/off_eu.parquet` (105 MB, 2.57M products)

## Key Format Discoveries
- `product_name`: list of structs `[{lang, text}]` — requires extraction of 'main' language
- `categories_tags`, `countries_tags`: numpy arrays, not Python lists
- `nutriments`: nested `STRUCT(name, value, 100g, ...)[]` — not flat columns
- `nutriscore_grade`: includes "not-applicable" (72K products) — treat as missing

## Pipeline Steps
1. Load OFF EU Parquet with `load_off_eu()` (handles product_name extraction, nutriscore cleaning)
2. Brand normalisation (lowercase, accent strip, first brand if comma-separated)
3. Deduplication on EAN code (only 53 duplicates — OFF is clean)
4. Category harmonisation: OFF `en:` tags → 2-level hierarchy (L1: broad group, L2: specific)
5. Private label flagging (80+ known PL brands across 15+ EU retailers)
6. Nutri-Score computation for products missing grades (vectorised)
7. Export cleaned dataset

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from src.data.load_off import load_off_eu, profile_off
from src.data.clean import clean_off_pipeline
from src.data.nutriscore import compute_nutriscore_column

In [ ]:
# Step 1: Load OFF EU data
df = load_off_eu()
print(f"Shape: {df.shape}")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1e6:.0f} MB")

# Initial profile
profile = profile_off(df)
print(f"\nNutri-Score coverage (official): {profile['nutriscore_coverage_pct']}%")
print(f"NOVA coverage: {profile['nova_coverage_pct']}%")
print(f"\nMissing values for key fields:")
for k, v in profile.items():
    if k.startswith("missing_") and k.endswith("_pct"):
        field = k.replace("missing_", "").replace("_pct", "")
        print(f"  {field}: {v}%")

## Data Quality Findings

| Metric | Value | Notes |
|--------|-------|-------|
| Total EU products | 2,568,322 | Filtered from 4.29M worldwide |
| Nutri-Score (official) | 31.8% | After removing "not-applicable" |
| Nutri-Score (after computation) | 59.3% | +706K computed from base nutrients |
| Brands present | 63.9% | 36.1% missing |
| Categories mappable | 42.4% | 51% lack `categories_tags` entirely |
| Fiber data | 25.2% | Limits Nutri-Score positive points computation |

### Nutri-Score Distribution (official only)
- E: 27% (worst) — D: 26% — C: 21% — A: 14% — B: 12%
- Heavy D+E skew (53%) suggests selection bias: products with health claims more likely to display scores

### Country Distribution
- France: 1.2M (47%) — Germany: 386K (15%) — Spain: 350K (14%) — Italy: 263K (10%) — Belgium: 96K (4%)

In [ ]:
# Step 4: Save cleaned dataset
output_path = Path("data/processed/off_eu_clean.parquet")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_parquet(output_path, index=False)
print(f"Saved {len(df):,} products to {output_path}")
print(f"File size: {output_path.stat().st_size / 1e6:.1f} MB")

In [ ]:
# Step 3: Compute Nutri-Score for products missing grades
df = compute_nutriscore_column(df)
print(f"\nFinal Nutri-Score distribution:")
print(df["nutriscore_grade"].value_counts(dropna=False).to_string())
print(f"\nCoverage: {df['nutriscore_grade'].notna().mean()*100:.1f}%")
print(f"Of which computed: {df['nutriscore_computed'].sum():,}")

In [ ]:
# Step 2: Run cleaning pipeline (brand normalisation, dedup, categories, PL flagging)
df = clean_off_pipeline(df)
print(f"After cleaning: {df.shape}")
print(f"\nCategory L1 distribution (top 10):")
print(df["category_l1"].value_counts().head(10).to_string())
print(f"\nPL products: {df['is_private_label'].sum():,} ({df['is_private_label'].mean()*100:.1f}%)")